In [83]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/processed-vietnamese-online-news/processed_news_dataset.csv
/kaggle/input/vietnamese-online-news-dataset/news_dataset.json


In [84]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import re

# Load dữ liệu

In [85]:
file_path = "/kaggle/input/vietnamese-online-news-dataset/news_dataset.json"
with open(file_path, "r", encoding="utf-8", errors="ignore") as f:
    data_json = f.read()

In [86]:
parsed = json.loads(data_json)

In [87]:
news_df = pd.DataFrame(parsed)
news_df_original = news_df.copy()

In [88]:
news_df.head()

,id,author,content,picture_count,processed,source,title,topic,url,crawled_at
0,218270,,"Chiều 31/7, Công an tỉnh Thừa Thiên - Huế đã c...",3,0,docbao.vn,"Tên cướp tiệm vàng tại Huế là đại uý công an, ...",Pháp luật,https://docbao.vn/phap-luat/ten-cuop-tiem-vang...,2022-08-01 09:09:22.817308
1,218269,(Nguồn: Sina),"Gần đây, Thứ trưởng Bộ Phát triển Kỹ thuật số,...",1,0,vtc.vn,"Bỏ qua mạng 5G, Nga tiến thẳng từ 4G lên 6G",Sống kết nối,https://vtc.vn/bo-qua-mang-5g-nga-tien-thang-t...,2022-08-01 09:09:21.181469
2,218268,Hồ Sỹ Anh,Kết quả thi tốt nghiệp THPT năm 2022 cho thấy ...,3,0,thanhnien.vn,Địa phương nào đứng đầu cả nước tổng điểm 3 mô...,Giáo dục,https://thanhnien.vn/dia-phuong-nao-dung-dau-c...,2022-08-01 09:09:15.311901
3,218267,Ngọc Ánh,Thống đốc Kentucky Andy Beshear hôm 31/7 cho h...,1,0,vnexpress,Người chết trong mưa lũ 'nghìn năm có một' ở M...,Thế giới,https://vnexpress.net/nguoi-chet-trong-mua-lu-...,2022-08-01 09:09:02.211498
4,218266,HẢI YẾN - MINH LÝ,Vụ tai nạn giao thông liên hoàn trên phố đi bộ...,12,0,soha,"Hải Phòng: Hình ảnh xe ""điên"" gây tai nạn liên...",Thời sự - Xã hội,https://soha.vn/hai-phong-hinh-anh-xe-dien-gay...,2022-08-01 09:09:01.601170


# Khám phá dữ liệu

In [89]:
news_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 184539 entries, 0 to 184538
Data columns (total 10 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   id             184539 non-null  int64 
 1   author         184539 non-null  object
 2   content        184539 non-null  object
 3   picture_count  184539 non-null  int64 
 4   processed      184539 non-null  int64 
 5   source         184534 non-null  object
 6   title          184539 non-null  object
 7   topic          184539 non-null  object
 8   url            184539 non-null  object
 9   crawled_at     184125 non-null  object
dtypes: int64(3), object(7)
memory usage: 14.1+ MB


In [90]:
news_df['id'].nunique()

184539

### Thông tin về tập dữ liệu
- Tập dữ liệu gồm 184539 bài báo tiếng việt.
- Mỗi bài báo có 10 trường thông tin là ID, author (tác giả), content (nội dung), picture_count (số hình ảnh được sử dụng), processed, source (nguồn của bài báo), title (tên bài báo), topic (chủ đề), url, crawled_at (thời gian bắt đầu thu thập thông tin).
- Các trường source và crawled_at có chứa giá trị null, các trường còn lại có đủ dữ liệu.

In [91]:
news_df['topic'] = news_df['topic'].fillna("").str.lower()
news_df['topic'] = news_df['topic'].apply(lambda x: x.strip() if isinstance(x, str) else x)
topic_counts = news_df['topic'].value_counts()
print("Số lượng topic: ", len(topic_counts))
print('===========================')
print('THỐNG KÊ SỐ LƯỢNG BÀI BÁO THEO TỪNG TOPIC')
print(topic_counts)

Số lượng topic:  1603
THỐNG KÊ SỐ LƯỢNG BÀI BÁO THEO TỪNG TOPIC
topic
                                    39488
thế giới                            12347
thể thao                            11194
xã hội                              10140
pháp luật                            8179
                                    ...  
sea games 31                            1
yamaha x-max 300: 4,65 lít/100km        1
cđ than - khoáng sản                    1
phân tích - bình luận                   1
người việt tử tế                        1
Name: count, Length: 1603, dtype: int64


In [92]:
# Tổng số bài
total_articles = len(news_df)

# Số bài có topic = NaN
null_count = news_df['topic'].isna().sum()

# Số bài topic là chuỗi rỗng ""
empty_count = (news_df['topic'] == "").sum()

print("Tổng số bài:", total_articles)
print("Topic = NaN:", null_count)
print("Topic = \"\":", empty_count)

print("Tổng số bài KHÔNG có topic hợp lệ:", null_count + empty_count)


Tổng số bài: 184539
Topic = NaN: 0
Topic = "": 39488
Tổng số bài KHÔNG có topic hợp lệ: 39488


### Thông tin về các topics của các bài báo
- Số lượng topic là 1603 topics
- Có nhiều topics chỉ có 1 bài báo
- Có 39488 bài báo không có topic

# Text processing

## Chuẩn hóa chữ thường

### Chuẩn hóa cột 'title'

In [93]:
news_df['title'] = news_df['title'].str.lower()
news_df['title']

0         tên cướp tiệm vàng tại huế là đại uý công an, ...
1               bỏ qua mạng 5g, nga tiến thẳng từ 4g lên 6g
2         địa phương nào đứng đầu cả nước tổng điểm 3 mô...
3         người chết trong mưa lũ 'nghìn năm có một' ở m...
4         hải phòng: hình ảnh xe "điên" gây tai nạn liên...
                                ...                        
184534                          nghi can sát hại cha bị bắt
184535              ông tất thành cang được đề nghị giảm án
184536    nga tuyên bố 'giải phóng' hoàn toàn khu dân cư...
184537                hlv malaysia dọa ngáng đường việt nam
184538                           khi giáo dục thành đối đầu
Name: title, Length: 184539, dtype: object

### Chuẩn hóa cột content

In [94]:
news_df['content'] = news_df['content'].str.lower()
news_df['content']

0         chiều 31/7, công an tỉnh thừa thiên - huế đã c...
1         gần đây, thứ trưởng bộ phát triển kỹ thuật số,...
2         kết quả thi tốt nghiệp thpt năm 2022 cho thấy ...
3         thống đốc kentucky andy beshear hôm 31/7 cho h...
4         vụ tai nạn giao thông liên hoàn trên phố đi bộ...
                                ...                        
184534    chiều 7/6, khoa bị công an quận phú nhuận phối...
184535    ngày 7/6, phiên xử phúc thẩm ông tất thành can...
184536    "các khu dân cư của thành phố severodonetsk đã...
184537    *việt nam - malaysia: 20h thứ tư 8/6, trên vne...
184538    tôi phải lập tức đưa qua nhà cô giáo ngữ văn l...
Name: content, Length: 184539, dtype: object

### Chuẩn hóa cột 'author'

In [95]:
news_df['author'] = news_df['author'].str.lower()
news_df['author']

0                          
1             (nguồn: sina)
2                 hồ sỹ anh
3                  ngọc ánh
4         hải yến - minh lý
                ...        
184534           quốc thắng
184535              thảo mi
184536            thanh tâm
184537             lâm thỏa
184538            lang minh
Name: author, Length: 184539, dtype: object

## Chuẩn hóa url trong bài báo

In [96]:
def normalize_urls(text):
    if not isinstance(text, str):
        return text
    # Tìm và thay thế URL bằng token <url>
    return re.sub(r'https?://\S+', '<url>', text)

# Áp dụng cho cột content
news_df['content'] = news_df['content'].apply(normalize_urls)

In [97]:
news_df['content'].iloc[0]

"chiều 31/7, công an tỉnh thừa thiên - huế đã có thông tin ban đầu về vụ nổ súng,cướp tiệm vàng tại chợ đông ba nằm trên đường trần hưng đạo (tp huế, tỉnh thừa thiên - huế). thông sài gòn giải phóng, khoảng 12h30' ngày 31/7, một đối tượng sử dụng súng ak bất ngờ xông vào tiệm vàng hoàng đức và thái lợi (phía trước chợ đông ba) rồi nổ súng chỉ thiên liên tiếp uy hiếp chủ tiệm để cướp vàng. sau đó, đối tượng mang số vàng vừa cướp được vứt ra vỉa hè rồi đi bộ đến khu vực cầu gia hội, cách khu vực gây án khoảng 300m. giám đốc công an tỉnh thừa thiên – huế lập tức trực tiếp chỉ đạo các lực lượng chức năng gồm công an tỉnh và công an tp huế nhanh chóng có mặt tại hiện trường triển khai đồng bộ các biện pháp nghiệp vụ, khoanh vùng và ngăn không để người dân đi vào hiện trường. hàng trăm tiểu thương trong chợ đông ba và người dân gần cầu gia hội được yêu cầu di chuyển khỏi hiện trường, đóng cửa nhà đề phòng đạn lạc. tuy nhiên, thấy vàng bị ném ra đường, nhiều người đua nhau nhặt, tạo cảnh nhốn

## Loại bỏ các ký tự không cần thiết

In [98]:
def clean_text(text):
    if not isinstance(text, str):
        return text
    text = text.lower()
    text = re.sub(r'[^a-zA-Z0-9À-ỹ\s\.,!?;:/-]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    text = re.sub(r'\s+(?=[\.,;:!)?])', '', text)
    text = re.sub(r'(?<=[\.,;:!)?])\s*(?=[\wÀ-ỹ])', ' ', text)
    text = re.sub(r'(?<=\d\.)\s+(?=\d)', '', text) # Xóa dấu cách thừa trong sđt
    return text

In [99]:
news_df['title'] = news_df['title'].apply(clean_text)
news_df['content'] = news_df['content'].apply(clean_text)
news_df['author'] = news_df['author'].apply(clean_text)

In [100]:
news_df['content'][0]

'chiều 31/7, công an tỉnh thừa thiên - huế đã có thông tin ban đầu về vụ nổ súng, cướp tiệm vàng tại chợ đông ba nằm trên đường trần hưng đạo tp huế, tỉnh thừa thiên - huế. thông sài gòn giải phóng, khoảng 12h30 ngày 31/7, một đối tượng sử dụng súng ak bất ngờ xông vào tiệm vàng hoàng đức và thái lợi phía trước chợ đông ba rồi nổ súng chỉ thiên liên tiếp uy hiếp chủ tiệm để cướp vàng. sau đó, đối tượng mang số vàng vừa cướp được vứt ra vỉa hè rồi đi bộ đến khu vực cầu gia hội, cách khu vực gây án khoảng 300m. giám đốc công an tỉnh thừa thiên huế lập tức trực tiếp chỉ đạo các lực lượng chức năng gồm công an tỉnh và công an tp huế nhanh chóng có mặt tại hiện trường triển khai đồng bộ các biện pháp nghiệp vụ, khoanh vùng và ngăn không để người dân đi vào hiện trường. hàng trăm tiểu thương trong chợ đông ba và người dân gần cầu gia hội được yêu cầu di chuyển khỏi hiện trường, đóng cửa nhà đề phòng đạn lạc. tuy nhiên, thấy vàng bị ném ra đường, nhiều người đua nhau nhặt, tạo cảnh nhốn nháo 

## Chuẩn hóa Unicode

In [101]:
import unicodedata

def normalize_unicode(text):
    if not isinstance(text, str):
        return text
    # NFC: chuẩn hóa về dạng ký tự đầy đủ (preferred cho tiếng Việt)
    return unicodedata.normalize("NFC", text)
news_df['title'] = news_df['title'].apply(normalize_unicode)
news_df['content'] = news_df['content'].apply(normalize_unicode)
news_df['author'] = news_df['author'].apply(normalize_unicode)

In [102]:
news_df['content'][0]

'chiều 31/7, công an tỉnh thừa thiên - huế đã có thông tin ban đầu về vụ nổ súng, cướp tiệm vàng tại chợ đông ba nằm trên đường trần hưng đạo tp huế, tỉnh thừa thiên - huế. thông sài gòn giải phóng, khoảng 12h30 ngày 31/7, một đối tượng sử dụng súng ak bất ngờ xông vào tiệm vàng hoàng đức và thái lợi phía trước chợ đông ba rồi nổ súng chỉ thiên liên tiếp uy hiếp chủ tiệm để cướp vàng. sau đó, đối tượng mang số vàng vừa cướp được vứt ra vỉa hè rồi đi bộ đến khu vực cầu gia hội, cách khu vực gây án khoảng 300m. giám đốc công an tỉnh thừa thiên huế lập tức trực tiếp chỉ đạo các lực lượng chức năng gồm công an tỉnh và công an tp huế nhanh chóng có mặt tại hiện trường triển khai đồng bộ các biện pháp nghiệp vụ, khoanh vùng và ngăn không để người dân đi vào hiện trường. hàng trăm tiểu thương trong chợ đông ba và người dân gần cầu gia hội được yêu cầu di chuyển khỏi hiện trường, đóng cửa nhà đề phòng đạn lạc. tuy nhiên, thấy vàng bị ném ra đường, nhiều người đua nhau nhặt, tạo cảnh nhốn nháo 

## Tách câu và từ dùng cho tiếng việt

### Dùng regrex

In [103]:
def vietnamese_sent_tokenize(text):
    # Tách theo ., ?, ! và giữ lại dấu câu
    sentences = re.split(r'(?<=[.!?])\s+', text)
    return [s.strip() for s in sentences if s.strip()]

In [104]:
sent_tokens = news_df['content'].apply(vietnamese_sent_tokenize)

In [105]:
len(sent_tokens.iloc[0])

22

In [106]:
type(sent_tokens)

pandas.core.series.Series

### Dùng thư viện underthesea

In [107]:
!pip install underthesea

In [108]:
from underthesea import sent_tokenize, word_tokenize

In [109]:
sent_tokens = news_df["content"].apply(lambda x: sent_tokenize(x))

In [110]:
len(sent_tokens.iloc[0])

22

In [111]:
type(sent_tokens)

pandas.core.series.Series

In [112]:
news_df['sent_tokens_content'] = sent_tokens

## Tách từ

In [113]:
tokenized_texts = []
for i, text in enumerate(news_df["content"]):
    tokens = word_tokenize(text, format="list")
    tokenized_texts.append(tokens)
    if i % 1000 == 0:
        print(f"Đã xử lý {i} văn bản...")
# news_df["tokens"] = tokenized_texts

Đã xử lý 0 văn bản...
Đã xử lý 1000 văn bản...
Đã xử lý 2000 văn bản...
Đã xử lý 3000 văn bản...
Đã xử lý 4000 văn bản...
Đã xử lý 5000 văn bản...
Đã xử lý 6000 văn bản...
Đã xử lý 7000 văn bản...
Đã xử lý 8000 văn bản...
Đã xử lý 9000 văn bản...
Đã xử lý 10000 văn bản...
Đã xử lý 11000 văn bản...


KeyboardInterrupt: 

In [114]:
word_tokens = [[t.replace(" ", "_") for t in token] for token in tokenized_texts]

In [115]:
news_df['word_tokens_content'] = None

In [116]:
news_df.loc[:len(word_tokens)-1, "word_tokens_content"] = pd.Series(word_tokens, dtype="object")

In [117]:
news_df['word_tokens_content'][0]

['chiều',
 '31/7',
 ',',
 'công_an',
 'tỉnh',
 'thừa_thiên',
 '-',
 'huế',
 'đã',
 'có',
 'thông_tin',
 'ban_đầu',
 'về',
 'vụ',
 'nổ_súng',
 ',',
 'cướp',
 'tiệm',
 'vàng',
 'tại',
 'chợ',
 'đông',
 'ba',
 'nằm',
 'trên',
 'đường',
 'trần_hưng_đạo',
 'tp',
 'huế',
 ',',
 'tỉnh',
 'thừa_thiên',
 '-',
 'huế',
 '.',
 'thông_sài',
 'gòn',
 'giải_phóng',
 ',',
 'khoảng',
 '12',
 'h30',
 'ngày',
 '31/7',
 ',',
 'một',
 'đối_tượng',
 'sử_dụng',
 'súng',
 'ak',
 'bất_ngờ',
 'xông',
 'vào',
 'tiệm',
 'vàng',
 'hoàng_đức',
 'và',
 'thái_lợi',
 'phía',
 'trước',
 'chợ',
 'đông',
 'ba',
 'rồi',
 'nổ_súng',
 'chỉ_thiên',
 'liên_tiếp',
 'uy_hiếp',
 'chủ',
 'tiệm',
 'để',
 'cướp',
 'vàng',
 '.',
 'sau',
 'đó',
 ',',
 'đối_tượng',
 'mang',
 'số',
 'vàng',
 'vừa',
 'cướp',
 'được',
 'vứt',
 'ra',
 'vỉa_hè',
 'rồi',
 'đi',
 'bộ',
 'đến',
 'khu_vực',
 'cầu',
 'gia_hội',
 ',',
 'cách',
 'khu_vực',
 'gây',
 'án',
 'khoảng',
 '300',
 'm',
 '.',
 'giám_đốc',
 'công_an',
 'tỉnh',
 'thừa',
 'thiên_huế',
 'lập

### Loại bỏ các dấu câu

In [118]:
import string

def remove_punctuation(tokens):
    if not isinstance(tokens, list):
        return tokens
    # Tập hợp dấu câu chuẩn
    punct = set(string.punctuation)
    punct.discard("_")
    # Bổ sung thêm dấu câu
    punct.update(['…', '“', '”', '‘', '’', '–'])
    # Giữ token không thuộc dấu câu và không phải "_" đơn lẻ
    return [tok for tok in tokens if tok not in punct and tok != "_"]


In [119]:
news_df['word_tokens_content'] = news_df['word_tokens_content'].apply(remove_punctuation)

In [120]:
news_df.to_csv("processed_news_dataset.csv", index=False, encoding="utf-8")

In [121]:
import ast
import pandas as pd
news_df = pd.read_csv('/kaggle/working/processed_news_dataset.csv')
news_df["word_tokens_content"] = news_df["word_tokens_content"].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) else []
)

/tmp/ipykernel_36/2264500344.py:3: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  news_df = pd.read_csv('/kaggle/working/processed_news_dataset.csv')


In [122]:
news_df.head()

,id,author,content,picture_count,processed,source,title,topic,url,crawled_at,sent_tokens_content,word_tokens_content
0,218270,NaN,"chiều 31/7, công an tỉnh thừa thiên - huế đã c...",3,0,docbao.vn,"tên cướp tiệm vàng tại huế là đại uý công an, ...",pháp luật,https://docbao.vn/phap-luat/ten-cuop-tiem-vang...,2022-08-01 09:09:22.817308,"['chiều 31/7, công an tỉnh thừa thiên - huế đã...","[chiều, 31/7, công_an, tỉnh, thừa_thiên, huế, ..."
1,218269,nguồn: sina,"gần đây, thứ trưởng bộ phát triển kỹ thuật số,...",1,0,vtc.vn,"bỏ qua mạng 5g, nga tiến thẳng từ 4g lên 6g",sống kết nối,https://vtc.vn/bo-qua-mang-5g-nga-tien-thang-t...,2022-08-01 09:09:21.181469,"['gần đây, thứ trưởng bộ phát triển kỹ thuật s...","[gần, đây, thứ_trưởng, bộ, phát_triển, kỹ_thuậ..."
2,218268,hồ sỹ anh,kết quả thi tốt nghiệp thpt năm 2022 cho thấy ...,3,0,thanhnien.vn,địa phương nào đứng đầu cả nước tổng điểm 3 mô...,giáo dục,https://thanhnien.vn/dia-phuong-nao-dung-dau-c...,2022-08-01 09:09:15.311901,['kết quả thi tốt nghiệp thpt năm 2022 cho thấ...,"[kết_quả, thi, tốt_nghiệp, thpt, năm, 2022, ch..."
3,218267,ngọc ánh,thống đốc kentucky andy beshear hôm 31/7 cho h...,1,0,vnexpress,người chết trong mưa lũ nghìn năm có một ở mỹ ...,thế giới,https://vnexpress.net/nguoi-chet-trong-mua-lu-...,2022-08-01 09:09:02.211498,['thống đốc kentucky andy beshear hôm 31/7 cho...,"[thống_đốc, kentucky, andy, beshear, hôm, 31/7..."
4,218266,hải yến - minh lý,vụ tai nạn giao thông liên hoàn trên phố đi bộ...,12,0,soha,hải phòng: hình ảnh xe điên gây tai nạn liên h...,thời sự - xã hội,https://soha.vn/hai-phong-hinh-anh-xe-dien-gay...,2022-08-01 09:09:01.601170,['vụ tai nạn giao thông liên hoàn trên phố đi ...,"[vụ, tai_nạn, giao_thông, liên_hoàn, trên, phố..."


## Bags of word

### Unigram

In [123]:
news_df["word_tokens_joined"] = news_df["word_tokens_content"].apply(
    lambda x: " ".join(x) if isinstance(x, list) else ""
)

In [124]:
news_df["word_tokens_joined"][0]

'chiều 31/7 công_an tỉnh thừa_thiên huế đã có thông_tin ban_đầu về vụ nổ_súng cướp tiệm vàng tại chợ đông ba nằm trên đường trần_hưng_đạo tp huế tỉnh thừa_thiên huế thông_sài gòn giải_phóng khoảng 12 h30 ngày 31/7 một đối_tượng sử_dụng súng ak bất_ngờ xông vào tiệm vàng hoàng_đức và thái_lợi phía trước chợ đông ba rồi nổ_súng chỉ_thiên liên_tiếp uy_hiếp chủ tiệm để cướp vàng sau đó đối_tượng mang số vàng vừa cướp được vứt ra vỉa_hè rồi đi bộ đến khu_vực cầu gia_hội cách khu_vực gây án khoảng 300 m giám_đốc công_an tỉnh thừa thiên_huế lập_tức trực_tiếp chỉ_đạo các lực_lượng chức_năng gồm công_an tỉnh và công_an tp huế nhanh_chóng có_mặt tại hiện_trường triển_khai đồng_bộ các biện_pháp nghiệp_vụ khoanh vùng và ngăn không để người_dân đi vào hiện_trường hàng trăm tiểu_thương trong chợ đông ba và người_dân gần cầu gia_hội được yêu_cầu di_chuyển khỏi hiện_trường đóng_cửa nhà đề_phòng đạn_lạc tuy_nhiên thấy vàng bị ném ra đường nhiều người đua nhau nhặt tạo cảnh nhốn_nháo trước cổng chợ do đ

In [125]:
from sklearn.feature_extraction.text import CountVectorizer

unigram = CountVectorizer(ngram_range = (1,1))
X_uni = unigram.fit_transform(news_df["word_tokens_joined"])

In [126]:
unigram.get_feature_names_out().shape

(75233,)

In [127]:
analyze = unigram.build_analyzer()
analyze("Ngày hôm nay, tôi thấy rất vui ạ.")

['ngày', 'hôm', 'nay', 'tôi', 'thấy', 'rất', 'vui']

In [128]:
X_uni.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [129]:
unigram.vocabulary_.get('hoa_cúc')

23503

### Bigram

In [130]:
bigram = CountVectorizer(ngram_range=(2,2))
X_bi = bigram.fit_transform(news_df["word_tokens_joined"])

In [131]:
bigram.get_feature_names_out()

array(['00 00', '00 20', '00 21', ..., 'ỷ_vân có', 'ỷ_vân ấn', 'ỷ_y vào'],
      dtype=object)

In [132]:
analyze = bigram.build_analyzer()
analyze("Ngày hôm nay, tôi thấy rất vui_vẻ ạ.")

['ngày hôm', 'hôm nay', 'nay tôi', 'tôi thấy', 'thấy rất', 'rất vui_vẻ']

In [133]:
bigram.vocabulary_.get('ngày hôm_nay')

517858

## TF-IDF

In [134]:
from sklearn.feature_extraction.text import TfidfTransformer

# Unigram
tfidf_uni = TfidfTransformer()
X_tfidf_uni = tfidf_uni.fit_transform(X_uni)

# Bigram
# X_tfidf_bi = tfidf_bi.fit_transform(X_bi)


In [135]:
X_tfidf_uni.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [136]:
df_tfidf = pd.DataFrame(X_tfidf_uni.toarray(), columns=unigram.get_feature_names_out())

In [139]:
df_tfidf.head()

,00,000,0000_điểm,000_90,000_binh_sĩ,000_bản,000_bảng,000_ca,000_chiến_binh,000_cđv,...,ửng,ự_nhiên,ựa,ỷ_lại,ỷ_lợi,ỷ_mình,ỷ_thiên_đồ,ỷ_vào,ỷ_vân,ỷ_y
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
